In [30]:
# Dependencies and Setup
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import sem



# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
mouse_df = pd.read_csv("mouse_drug_data.csv")
mouse_df.head()

,Mouse ID,Drug
0,f234,Stelasyn
1,x402,Stelasyn
2,a492,Stelasyn
3,w540,Stelasyn
4,v764,Stelasyn


In [31]:
clinical_df = pd.read_csv("clinicaltrial_data.csv")

clinical_df.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [32]:
# Combine the data into a single dataset
combined_mouse_df = pd.merge(mouse_df, clinical_df, on="Mouse ID")
combined_mouse_df.head()

# Display the data table for preview


,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0
3,f234,Stelasyn,15,48.735197,1
4,f234,Stelasyn,20,51.112713,2


In [4]:
# Combine the data into a single dataset


# Display the data table for preview

## Tumor Response to Treatment

In [33]:
# Store the Mean Tumor Volume Data Grouped by Drug and Timepoint 
Tumor_group = combined_mouse_df.groupby(['Drug', 'Timepoint'])
Tumor_group.head()

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0
3,f234,Stelasyn,15,48.735197,1
4,f234,Stelasyn,20,51.112713,2
...,...,...,...,...,...
1780,x773,Placebo,35,55.192736,4
1781,x773,Placebo,40,55.987676,4
1782,x773,Placebo,45,58.634971,4
1802,y478,Placebo,40,65.971120,1


In [34]:
average_tumor_group = Tumor_group[['Tumor Volume (mm3)']].mean()
average_tumor_group = average_tumor_group.rename(columns={"Tumor Volume (mm3)":"Average Tumor Volume (mm3)"})
average_tumor_group.head()
# Convert to DataFrame

# Preview DataFrame

Average Tumor Volume (mm3)
Drug      Timepoint                            
Capomulin 0                           45.000000
          5                           44.266086
          10                          43.084291
          15                          42.064317
          20                          40.716325

In [35]:
# Store the Standard Error of Tumor Volumes Grouped by Drug and Timepoint
sem_tumor_group = Tumor_group[['Tumor Volume (mm3)']].sem()
sem_tumor_group = sem_tumor_group.rename(columns={"Tumor Volume (mm3)":"Sem Average Tumor Volume"})
sem_tumor_group.head()

# Convert to DataFrame

# Preview DataFrame



Sem Average Tumor Volume
Drug      Timepoint                          
Capomulin 0                          0.000000
          5                          0.448593
          10                         0.702684
          15                         0.838617
          20                         0.909731

In [40]:
# Minor Data Munging to Re-Format the Data Frames
combined_tumor_df = pd.merge(average_tumor_group, sem_tumor_group, on=("Drug", "Timepoint"))
combined_tumor_df.head()
# Preview that Reformatting worked


Average Tumor Volume (mm3)  Sem Average Tumor Volume
Drug      Timepoint                                                      
Capomulin 0                           45.000000                  0.000000
          5                           44.266086                  0.448593
          10                          43.084291                  0.702684
          15                          42.064317                  0.838617
          20                          40.716325                  0.909731

In [36]:
# Generate the Plot (with Error Bars)
fig, ax=plt.subplots()
ax.errorbar(np.arange(0, len(average_tumor_group)), average_tumor_group, yerr = sem_tumor_group, fmt="o", label="Average Tumor Volume")
# Save the Figure



<IPython.core.display.Javascript object>

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
# Show the Figure
plt.show()

![Tumor Response to Treatment](../Images/treatment.png)

## Metastatic Response to Treatment

In [ ]:
# Store the Mean Met. Site Data Grouped by Drug and Timepoint 

# Convert to DataFrame

# Preview DataFrame


In [ ]:
# Store the Standard Error associated with Met. Sites Grouped by Drug and Timepoint 

# Convert to DataFrame

# Preview DataFrame


In [ ]:
# Minor Data Munging to Re-Format the Data Frames

# Preview that Reformatting worked


In [ ]:
# Generate the Plot (with Error Bars)

# Save the Figure

# Show the Figure


![Metastatic Spread During Treatment](../Images/spread.png)

## Survival Rates

In [ ]:
# Store the Count of Mice Grouped by Drug and Timepoint (W can pass any metric)

# Convert to DataFrame

# Preview DataFrame


In [ ]:
# Minor Data Munging to Re-Format the Data Frames

# Preview the Data Frame


In [ ]:
# Generate the Plot (Accounting for percentages)

# Save the Figure

# Show the Figure
plt.show()

![Metastatic Spread During Treatment](../Images/survival.png)

## Summary Bar Graph

In [ ]:
# Calculate the percent changes for each drug

# Display the data to confirm


In [ ]:
# Store all Relevant Percent Changes into a Tuple


# Splice the data between passing and failing drugs


# Orient widths. Add labels, tick marks, etc. 


# Use functions to label the percentages of changes


# Call functions to implement the function calls


# Save the Figure


# Show the Figure
fig.show()

![Metastatic Spread During Treatment](../Images/change.png)